In [1]:
from pro.data.vocdata import VocDataset
label_dir = 'pro/data/dataset/VOCdevkit/VOC2007/Annotations'
image_dir = 'pro/data/dataset/VOCdevkit/VOC2007/JPEGImages'
voc_dataset = VocDataset(label_dir, image_dir)


In [2]:
label_dict = voc_dataset.dataset_dict
label_dict

{'007826': [[500.0, 375.0, 80.0, 217.0, 320.0, 273.0, 0],
  [500.0, 375.0, 197.0, 193.0, 257.0, 326.0, 1],
  [500.0, 375.0, 139.0, 184.0, 185.0, 231.0, 1],
  [500.0, 375.0, 258.0, 180.0, 312.0, 314.0, 1],
  [500.0, 375.0, 10.0, 195.0, 93.0, 358.0, 1],
  [500.0, 375.0, 82.0, 252.0, 243.0, 372.0, 1],
  [500.0, 375.0, 43.0, 319.0, 144.0, 375.0, 1]],
 '002786': [[500.0, 332.0, 80.0, 97.0, 348.0, 272.0, 2],
  [500.0, 332.0, 201.0, 52.0, 258.0, 202.0, 3]],
 '006286': [[500.0, 375.0, 80.0, 88.0, 405.0, 375.0, 3],
  [500.0, 375.0, 436.0, 147.0, 475.0, 209.0, 3],
  [500.0, 375.0, 381.0, 145.0, 428.0, 191.0, 3],
  [500.0, 375.0, 402.0, 219.0, 500.0, 375.0, 0],
  [500.0, 375.0, 347.0, 177.0, 405.0, 216.0, 0],
  [500.0, 375.0, 160.0, 167.0, 203.0, 188.0, 0],
  [500.0, 375.0, 104.0, 185.0, 209.0, 279.0, 0],
  [500.0, 375.0, 1.0, 322.0, 82.0, 375.0, 0],
  [500.0, 375.0, 2.0, 169.0, 53.0, 225.0, 0],
  [500.0, 375.0, 408.0, 241.0, 500.0, 375.0, 1],
  [500.0, 375.0, 49.0, 178.0, 135.0, 279.0, 1],
  [50

In [11]:
class_dict = {}
for i, classname in enumerate(voc_dataset.classes): 
    class_dict[i] = classname
print(class_dict)

{0: 'diningtable', 1: 'chair', 2: 'horse', 3: 'person', 4: 'tvmonitor', 5: 'bird', 6: 'cow', 7: 'dog', 8: 'bottle', 9: 'pottedplant', 10: 'aeroplane', 11: 'car', 12: 'cat', 13: 'sheep', 14: 'bicycle', 15: 'sofa', 16: 'boat', 17: 'train', 18: 'motorbike', 19: 'bus'}


In [4]:
image, target,image_id = voc_dataset.__getitem__(2)
print(target)

{'boxes': tensor([[ 80.,  88., 405., 375.],
        [436., 147., 475., 209.],
        [381., 145., 428., 191.],
        [402., 219., 500., 375.],
        [347., 177., 405., 216.],
        [160., 167., 203., 188.],
        [104., 185., 209., 279.],
        [  1., 322.,  82., 375.],
        [  2., 169.,  53., 225.],
        [408., 241., 500., 375.],
        [ 49., 178., 135., 279.],
        [135., 164., 167., 188.],
        [150., 184., 233., 290.],
        [ 19., 167.,  65., 231.],
        [ 56., 163.,  95., 209.],
        [ 97., 177., 152., 203.],
        [184., 166., 231., 194.],
        [196., 181., 240., 208.],
        [400., 178., 454., 232.],
        [  1., 169.,  32., 174.]]), 'labels': tensor([3, 3, 3, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'area': tensor([93275.,  2418.,  2162., 15288.,  2262.,   903.,  9870.,  4293.,  2856.,
        12328.,  8686.,   768.,  8798.,  2944.,  1794.,  1430.,  1316.,  1188.,
         2916.,   155.]), 'iscrowd': tensor([0, 0, 0, 0, 0, 

In [5]:
from pro.data.vocdata import VocDataloader
voc_dataloader= VocDataloader(voc_dataset)
train_dataloader, val_dataloader = voc_dataloader(0.7,batch_size_train=3)

In [8]:
batch = next(iter(train_dataloader))
image_batch, target_batch, image_id_batch= batch
images = list(image.to('cpu') for image in image_batch)
targets = [{k: v.to('cpu') for k, v in t.items()} for t in target_batch]

In [13]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT'
)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 21)
model.train()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [14]:
loss_dict = model(images, targets)
print(loss_dict)

In [27]:
loss_dict.keys()
# 

dict_keys(['loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg'])

In [22]:
print(loss_dict)
losses = sum(loss for loss in loss_dict.values())
print(losses)

{'loss_classifier': tensor(3.0732, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.2091, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0380, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0555, grad_fn=<DivBackward0>)}
tensor(3.3758, grad_fn=<AddBackward0>)


In [25]:
losses.backward()

In [26]:
losses


tensor(3.3758, grad_fn=<AddBackward0>)

In [32]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y%m%d-%H%M%S")
print(current_time)

164501


'16:44:38'